# X-ray diffraction (XRD) spectra trends

*Authors: Enze Chen (University of California, Berkeley)*

![Powder XRD spectra](https://raw.githubusercontent.com/enze-chen/learning_modules/master/fig/XRD_labeled.png)

This is an interactive notebook for playing around with some experimental parameters ($a$, $\lambda$, $T$, etc.) and observing the effect on the resulting XRD spectra. I find XRD to be a particularly beautiful subject and I couldn't find any similar visualizations online. I hope this interactive demo will help you learn the _qualitative trends_ associated with powder XRD spectra.

## Prerequisites

To get the most out of this notebook, you should already have:    
- Knowledge of XRD fundamentals such as Bragg's law and intensity factors.

## Learning goals

By the end of this notebook, you should be able to *assess* how changing the following experimental inputs affects the XRD spectra:     
- Crystal structure
- Lattice constant
- X-ray wavelength
- Temperature
- Strain
- Crystallite size

### Interested in coding?

If you were looking for a more thorough review, including a **scaffolded programming exercise** to generate the XRD spectra, please see [my other notebook](https://colab.research.google.com/github/enze-chen/learning_modules/blob/master/mse/XRD_plotting.ipynb) that will walk you through most of the details.

## How to use this notebook

If you are viewing this notebook on [Google Colaboratory](https://colab.research.google.com/github/enze-chen/learning_modules/blob/master/mse/XRD_trends.ipynb), then everything is already set up for you (hooray).
If you want to **save a copy** of this notebook for yourself, go to "File > Save a copy in Drive" and you will find it in your Google Drive account under "My Drive > Colab Notebooks."
If you want to run the notebook locally, you can download it and make sure all the Python modules in the [`requirements.txt`](https://github.com/enze-chen/learning_modules/blob/master/requirements.txt) file are installed before running it.

To run this notebook, run all the cells (e.g. `Runtime > Run all` in the menu) and then adjust the sliders at the bottom. Click `Generate Plot` each time you change a setting. I **strongly recommend** just running the code and experimenting with the inputs *before* reading the code in great detail.

---------------------------

## A few important equations (very quick review)

By far the most important equation is [not surprisingly] **Bragg's law**, given by 

$$n\lambda = 2d \sin(\theta)$$

where $n$ is the order (typically $1$), $\lambda$ is the wavelength, $d$ is the interplanar spacing, and $\theta$ is the Bragg angle. Here we will solve for $\theta$ as follows:

$$ \lambda = 2d \sin(\theta) \longrightarrow \theta = \sin^{-1} \left( \frac{\lambda}{2d} \right), \quad d = \frac{a}{\sqrt{h^2 + k^2 + l^2}} $$

where $h,k,l$ are the miller indices of the diffracting plane and $a$ is the lattice constant. The above formula for $d$ assumes a cubic structure.

Another important equation is for the **Intensity**, given by

$$ I = |F|^2 \times P \times L \times m \times A \times T $$

where
* $F$ is the structure factor (we take the modulus before squaring because it might be a complex number).
* $P$ is the polarization factor.
* $L$ is the Lorentz factor.
* $m$ is the multiplicity factor.
* $A$ is the absorption factor.
* $T$ is the temperature factor.

Furthermore, recall that size effects can be included through the **Scherrer equation**, given by 

$$ t = \frac{K\lambda}{\beta \cos(\theta)} $$ 

where $t$ is the crystallite/grain thickness, $K \sim 0.9$ is a shape factor, and $\beta$ is the full width at half maximum of the peak in radians.

For more information, please reference [*Elements of X-Ray Diffraction* (3rd)](https://www.pearson.com/us/higher-education/program/Cullity-Elements-of-X-Ray-Diffraction-3rd-Edition/PGM113710.html) by Cullity and Stock, which is a fantastic textbook.

----------------------------------

## Python module imports

These are all the required Python modules.

In [ ]:
# General modules
import itertools

# Scientific computing modules
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
%matplotlib inline

# Interactivity modules
from ipywidgets import interact_manual, fixed, \
                       IntSlider, FloatSlider, FloatLogSlider, RadioButtons, \
                       Button, Layout

## Widget function

Our widget will call `plot_XRD()` each time we interact with it. This function calculates the structure factor and the intensities and then plots the spectra on an $\text{Intensity vs. } 2\theta$ plot. I've tried my best to keep the code simple and yet illustrative. Please see comments for more information.

In [ ]:
def plot_XRD(a, wavelength, cell_type, thickness, T=0, K=0.94):
    """This function is called by the widget to perform the plotting based on inputs.
    
    Args:
        a (float): Lattice constant in nanometers.
        wavelength (float): X-ray wavelength in nanometers.
        cell_type (str): Crystal structure, can be FCC, BCC, or DC.
        thickness (float): Crystallite size in nanometers.
        T (int): Temperature in Kelvin. Default = 0.
        K (float): Scherrer equation parameter. Default = 0.94 (cubic).
        
    Returns:
        None, but a pyplot is displayed.
    """
    
    # Crystallographic planes
    planes = [[1,0,0], [1,1,0], [1,1,1], [2,0,0], [2,1,0], [2,1,1], [2,2,0],\
              [2,2,1], [3,0,0], [3,1,0], [3,1,1], [2,2,2], [3,2,0], [3,2,1]]
    planes_str = [f'$({p[0]}{p[1]}{p[2]})$' for p in planes]   # string labels

    # Set the basis
    basis = []
    if cell_type is 'FCC':
        basis = np.array([[0,0,0], [0.5,0.5,0], [0.5,0,0.5], [0,0.5,0.5]])
    elif cell_type is 'BCC':
        basis = np.array([[0,0,0], [0.5,0.5,0.5]])
    elif cell_type is 'DC':
        basis = np.array([[0,0,0], [0.5,0.5,0], [0.5,0,0.5], [0,0.5,0.5],
                          [0.25,0.25,0.25], [0.75,0.75,0.25], \
                          [0.75,0.25,0.75], [0.25,0.75,0.75]])
    else:
        raise ValueError('Cell type not yet supported.')

    # Convert planes to theta values (see equation above)
    s_vals = np.array([np.linalg.norm(p) for p in planes])
#     a += 1e-5 * T   # thermal expansion estimate; omit b/c a is alread indep var.
    theta = np.arcsin(np.divide(wavelength/2, np.divide(a, s_vals)))
    two_theta = 2 * np.degrees(theta)

    # Scherrer equation calculations
    beta = np.degrees(K * wavelength / thickness * np.divide(1, np.cos(theta)))
    sigma = beta / 2.355  # proportionality for Gaussian distribution

    # Structure-Temperature factor. Must... resist... for loops...
    s = np.sin(theta) / (10*wavelength)
    S = 2.210 * np.exp(-58.727*s**2) + 2.134 * np.exp(-13.553*s**2) + \
        1.689 * np.exp(-2.609*s**2) + 0.524 * np.exp(-0.339*s**2)
    f = 28 - 41.78214 * np.multiply(s**2, S)  # formula from Ch. 12 of De Graef
    F = np.multiply(f, np.sum(np.exp(2 * np.pi * 1j * \
                                     np.dot(np.array(planes), basis.T)), axis=1))

    # Multiplicity factor
    mult = [2**np.count_nonzero(p) * \
            len(set(itertools.permutations(p))) for p in planes]

    # Lorentz-Polarization factor
    Lp = np.divide(1 + np.cos(2 * theta)**2, 
                   np.multiply(np.sin(theta)**2, np.cos(theta)))

    # Final intensity
    I = np.multiply(np.absolute(F)**2, np.multiply(mult, Lp))
    
    # Plotting
    plt.rcParams.update({'figure.figsize':(10,5), 'font.size':22, 'axes.linewidth':2,
                         'mathtext.fontset':'cm'})
    xmin, xmax = (20, 160)
    x = np.linspace(xmin, xmax, int(10*(xmax-xmin)))
    fig, ax = plt.subplots()
    
    # Thermal effects. These functional dependencies ARE NOT REAL!!!
    thermal_diffuse = 3e1 * T * np.cbrt(x)   # background signal
    sigma += (T + 5) / 2000    # peak broadening from vibrations
    
    # Save all the curves, then take a max envelope
    all_curves = []
    for i in range(len(sigma)):
        y = stats.norm.pdf(x, two_theta[i], sigma[i])
        normed_curve = y / max(y) * I[i]
        # Don't include the curves that aren't selected by the Structure factor
        if max(normed_curve) > 1e1:
            max_ind = normed_curve.argmax()
            ax.annotate(s=planes_str[i], \
                        xy=(x[max_ind], normed_curve[max_ind] + thermal_diffuse[max_ind]))
            all_curves.append(normed_curve)
    final_curve = np.max(all_curves, axis=0) + thermal_diffuse
    plt.plot(x, final_curve, c='C0', lw=4, alpha=0.7)

    # Some fine-tuned settings for visual appeal
    for side in ['top', 'right']:
        ax.spines[side].set_visible(False)
    ax.set_xlim(xmin, xmax)
    ax.set_ylim(0, 1.05 * ax.get_ylim()[1])
    ax.tick_params(left=False, labelleft=False, direction='in', length=10, width=2)
    ax.set_xlabel(r'$2\theta$ (degree)')
    ax.set_ylabel('Intensity (a.u.)')
    plt.show()

Now we create each slider individually for readability and customization.

In [ ]:
b1 = Button(description='A button', layout=Layout(width='400px', height='30px'))
b2 = Button(description='A button', layout=Layout(width='400px', height='60px'))

a_widget = FloatSlider(value=0.352, min=0.31, max=0.4, step=0.001, 
                       description='Lattice constant (nm)', readout_format='.3f', 
                       style={'description_width':'150px'}, layout=b1.layout)

w_widget = FloatSlider(value=0.154, min=0.13, max=0.16, step=0.001, 
                       description='X-ray wavelength (nm)', readout_format='.3f', 
                       style={'description_width':'150px'}, layout=b1.layout)

c_widget = RadioButtons(options=['FCC', 'BCC', 'DC'], description='Crystal structure',
                        style={'description_width':'150px'}, layout=b2.layout)

t_widget = FloatLogSlider(value=10, base=10, min=0, max=3, step=0.1, 
                          description='Crystallite thickness (nm)',  readout_format='d', 
                          style={'description_width':'150px'}, layout=b1.layout)

T_widget = IntSlider(value=298, min=0, max=1000, step=1, 
                     description='Temperature (K)', readout_format='d', 
                     style={'description_width':'150px'}, layout=b1.layout)

After you have set your parameters, click **Generate Plot** to see the resulting XRD spectra.

In [ ]:
interact_manual.opts['manual_name'] = 'Generate Plot'
interact_manual(plot_XRD, a=a_widget, wavelength=w_widget, cell_type=c_widget, 
                          thickness=t_widget, T=T_widget, K=fixed(0.94));

--------------------------------------

## Discussion questions

* Can you rationalize all the trends you see? 
* Describe all the ways **temperature** affects the XRD spectra.
* How do we account for **strain** in our model? What differences might we observe between isotropic and anisotropic strain?
* If you're interested in scientific computing, try to understand how the structure factor ($F$) is calculated with clever [NumPy](https://numpy.org/) tools.

--------------------------------------

## Conclusion
I hope you found this notebook helpful in learning more about XRD and what affects a powder XRD spectra. 
Please don't hesitate to reach out if you have any questions or ideas to contribute.

## Acknowledgements

I thank Laura Armstrong, Nathan Bieberdorf, Han-Ming Hau, and Divya Ramakrishnan for user testing and helpful suggestions. 
I also thank [Prof. Andrew Minor](https://mse.berkeley.edu/people_new/minor/) for teaching MATSCI 204: Materials Characterization and my advisor [Prof. Mark Asta](https://mse.berkeley.edu/people_new/asta/) for his unwavering encouragement for my education-related pursuits. 
Interactivity is enabled with the [`ipywidgets`](https://ipywidgets.readthedocs.io/en/stable/) library. 
This project is generously hosted on [GitHub](https://github.com/enze-chen/learning_modules) and [Google Colaboratory](https://colab.research.google.com/github/enze-chen/learning_modules/blob/master/mse/XRD_trends.ipynb).

----------------------------------------

## Assumptions I've taken great liberties with

* For the Structure factor, I greatly oversimplified the construction of the atomic scattering factor ($f$) and selected some numbers from the data for Ni.
* I also combined part of the temperature factor into the structure factor. 
* I combined the Lorentz and polarization factors, as is commonly done in the literature.
* I ignored the absorption factor since it is more or less independent of $\theta$.
* I used a $\sqrt[3]{\theta}$ term to approximate the thermal background's general shape. I don't know the true analytical dependence, if there even is one.
* I used a Gaussian distribution to model each peak to capture crystallite size effects. Peaks in general are *not* Gaussian.

## Known issues

* It doesn't have great safeguards against numerical errors, such as invalid `arcsin` arguments and `NaN`. Please be gentle. ❤
* There's a weird rendering error where for large intensities the upper limit (e.g. `1e6`) appears on the y-axis. **:shrug:**
* I left out "simple cubic" as one of the candidate structures because it results in too many numerical instabilities to correct for. It's also a boring spectra.